In [202]:
import pandas as pd
import numpy as np
import chardet
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [203]:
df = pd.read_csv("ecom_yl.csv", sep=",", decimal='.')

In [204]:
#приводим названия столбцов в формат PEP8

df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns

Index(['user_id', 'region', 'device', 'channel', 'session_start',
       'session_end', 'sessiondurationsec', 'session_date', 'month', 'day',
       'hour_of_day', 'order_dt', 'revenue', 'payment_type', 'promo_code'],
      dtype='object')

In [205]:
def time(row):
    times = {
        'morning': [datetime.time(hour=6), datetime.time(hour=9, minute=59)],
        'day': [datetime.time(hour=10), datetime.time(hour=16, minute=59)],
        'evening': [datetime.time(hour=17), datetime.time(hour=21, minute=59)],
        'night': [datetime.time(hour=22), datetime.time(hour=5, minute=59)]
    }
    a = row.session_start.time
    b = row.session_end.time
    for i in times.keys:
        if a >= times[i][0] and b <= times[i][1]:
            return i
        if a >= times[i][0] and b > times[i][1]:
            if row.sessiondurationsec / 2 <= (times[i][1] - a).seconds:
                return i
        if a <= times[i][0] and b >= times[i][1]:
            return i
        if a < times[i][0] and b <= times[i][1]:
            if row.sessiondurationsec / 2 <= (b - times[i][0]).seconds:
                return i

In [206]:
#исправляем неправильные названия в region

dict_regions = {'United States':'United States', 'Frаnce':'France', 'Unjted States' : 'United States', 'Germany' : 'Germany', 
               'UK' : 'UK', 'France':'France', 'Frаncе': 'France', 'germany':'Germany', 'UК':'UK'}
df["region"] = df["region"].map(dict_regions)
df["region"].unique()

array(['United States', 'France', 'Germany', 'UK', nan], dtype=object)

In [207]:
#исправляем неправильные названия в channel

dict_channels = {'социальные сети' : 'социальные сети', 'organic':'organic', 'реклама у блогеров':'реклама у блогеров', 
                 'контексная реклама' : 'контекстная реклама', 'контекстная реклама':'контекстная реклама', 'email-рассылки':'email-рассылки'}
df["channel"] = df["channel"].map(dict_channels)
df["channel"].unique()

array(['социальные сети', 'organic', 'реклама у блогеров',
       'контекстная реклама', 'email-рассылки', nan], dtype=object)

In [208]:
#исправляем неправильные названия в device

dict_devices = {'iPhone' : 'iPhone', 'PC' : 'PC', 'Mac' : 'Mac', 'Android' : 'Android', 'android' : 'Android'}
df["device"] = df["device"].map(dict_devices)
df["device"].unique()

array(['iPhone', 'PC', 'Mac', 'Android', nan], dtype=object)

In [209]:
#убираем выбросы у SessionDuration с помощью 3-ного интерквартильного размаха

def transformed(x):
    i_quant = x.quantile(0.75) - x.quantile(0.25)
    k = x > (x.quantile(0.75) + i_quant * 3)
    x[k] = x.median()
    return x
    
df["sessiondurationsec"] = df.groupby("channel", group_keys=False)["sessiondurationsec"].transform(transformed)
df['revenue'] = df['revenue'].apply(lambda x: 4999 if (x == 100000) or (x == 1) else x)

In [210]:
#переводим даты из строкого формата в datetime

df['session_start'] = pd.to_datetime(df['session_start'])
df['session_end'] = pd.to_datetime(df['session_end'])

In [211]:
#заполняем пропуски в SessionDurationSec

df['sessiondurationsec'] = (df['session_end'] - df['session_start']).dt.total_seconds()

In [212]:
#Удаляем сессии длящиеся 5 секунд и меньше, в которых не было покупок

df = df[(df["sessiondurationsec"] > 5) | ((df["sessiondurationsec"] <= 5) & (df['revenue'].notna()))]

In [213]:
# число явных дубликатов и их удаление
i_see_it = len(df) - len(df.drop_duplicates())
#print(i_see_it) -> 3
df = df.drop_duplicates()

In [214]:
# заполнение пропусков
stupid_nans = df.groupby(['month'])['region']
df['region'] = df['region'].fillna(lambda x: stupid_nans[x])
stupid_nans = df.groupby(['month'])['device']
df['device'] = df['device'].fillna(lambda x: stupid_nans[x])
stupid_nans = df.groupby(['month'])['channel']
df['channel'] = df['channel'].fillna(lambda x: stupid_nans[x])

In [215]:
#очищение неправильных данных в promo_code
df['promo_code'] = df['promo_code'].map(lambda x: 1 if not pd.isna(x) and x else x)

In [216]:
#Добавление итоговой суммы покупки с учетом использования промокода
df['revenue_with_promo_code'] = df.apply(lambda x: 0.9 * x.revenue if x.promo_code == 1 and not pd.isna(x.revenue) else x.revenue, axis=1)

In [217]:
#Обозначение платящих пользователей
df['payer'] = df['revenue'].apply(lambda x: 1 if not pd.isna(x) else 0)

In [218]:
df['time_of_day'] = df.apply(time, axis=1)

TypeError: 'builtin_function_or_method' object is not iterable

In [221]:
df

,user_id,region,device,channel,session_start,session_end,sessiondurationsec,session_date,month,day,hour_of_day,order_dt,revenue,payment_type,promo_code,revenue_with_promo_code,payer
0,529697267522,United States,iPhone,социальные сети,2019-05-01 00:06:40,2019-05-01 00:07:06,26.0,2019-05-01,5,3,0,2019-05-01 00:06:40,9999.0,Mobile payments,0.0,9999.0,1
1,601292388085,United States,PC,organic,2019-05-01 06:56:16,2019-05-01 07:09:18,782.0,2019-05-01,5,3,7,NaN,NaN,NaN,NaN,NaN,0
2,852898876338,United States,Mac,социальные сети,2019-05-01 04:30:45,2019-05-01 04:34:56,251.0,2019-05-01,5,3,4,NaN,NaN,NaN,NaN,NaN,0
3,998513020664,United States,iPhone,социальные сети,2019-05-01 18:53:42,2019-05-01 18:57:35,233.0,2019-05-01,5,3,18,NaN,NaN,NaN,NaN,NaN,0
4,240702200943,United States,Mac,социальные сети,2019-05-02 14:04:32,2019-05-02 14:09:51,319.0,2019-05-02,5,4,14,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,246163171672,United States,iPhone,социальные сети,2019-10-29 18:12:24,2019-10-29 18:13:39,75.0,2019-10-29,10,2,18,NaN,NaN,NaN,NaN,NaN,0
999,660140862425,Germany,Android,реклама у блогеров,2019-10-30 19:14:02,2019-10-30 19:46:19,1937.0,2019-10-30,10,3,19,NaN,NaN,NaN,NaN,NaN,0
1000,898253063062,France,iPhone,organic,2019-10-30 03:18:22,2019-10-30 04:43:35,5113.0,2019-10-30,10,3,4,NaN,NaN,NaN,NaN,NaN,0
1001,780224321316,United States,iPhone,социальные сети,2019-10-31 22:56:17,2019-10-31 23:42:27,2770.0,2019-10-31,10,4,23,NaN,NaN,NaN,NaN,NaN,0


In [222]:
df.to_csv("res.csv", index=False)

In [223]:
df = pd.read_csv("res.csv", sep=",", decimal='.')